In [48]:
import os
import openai
import pandas as pd
import tiktoken
import numpy as np
from collections import Counter
from dotenv import load_dotenv
from openai import OpenAI
from langchain.schema import Document
from langchain_pinecone import PineconeVectorStore
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import Pinecone

In [49]:
# Fine tuning increases the accuracy from 84% to 84.20%; minor change

In [50]:
df = pd.read_csv('../../data/csv/toxic_classification_evaluation_set.csv')
df['Toxic'] = df['Toxic'].apply(lambda x: str(int(x)))
df

,Text,Toxic
0,largely written head problem put thoughts scre...,0
1,unable answer question7713019517,0
2,According edits add ratings based brothers poi...,0
3,take back Watch back u gwenrol whatever europe...,0
4,Take care believe bullshit see Bullshit 991842...,1
...,...,...
995,well know fuck idea notability fuck line make ...,1
996,mane fuck souja boi nigga dont go hard n paint,1
997,certainly better world service called simple N...,0
998,please interact leave warnings lie talk page c...,1


In [51]:
load_dotenv()
FINE_TUNED_TOXIC_DETECTION_API_KEY = os.getenv('FINE_TUNED_TOXIC_DETECTION_API_KEY')
pinecone_api_key = os.getenv('PINECONE_API_KEY')
pc_index = os.getenv('PINECONE_GPT')

In [52]:
fine_tuned = OpenAI(
    api_key=FINE_TUNED_TOXIC_DETECTION_API_KEY
)

In [53]:
# initializing Pinecone vector database instance
docSearch = Pinecone(
    index_name=pc_index,
    embedding=OpenAIEmbeddings(openai_api_key=FINE_TUNED_TOXIC_DETECTION_API_KEY)
)

In [54]:
temp_comment = "Classify these comments as either toxic (1) non-toxic (0). Toxic comments contain offensive, harmful, or abusive language. Label 1 for toxic and 0 for non-toxic \n\nExamples for Reference:"

In [55]:
# res = docSearch.max_marginal_relevance_search("please consider personal attack user Node eu Dear Node Eu take personal pride honor say commander well deserved Fuck yes know moment real identity please forgive wikipedia want assure family extend family mr Bortnikov harm intended untended occur long process well spend money involved realize finally deserve mercy Good look yes wikipedia last posting realize place muchmuch better real scientific world,0", k=25, fetch_k=10)
# text_one = res[1].page_content.split(' - ')
# text_two = res[2].page_content.split(' - ')
# text_three = res[3].page_content.split(' - ')
# text_four = res[4].page_content.split(' - ')
# text_five = res[5].page_content.split(' - ')

# temp_comment += '\n\n' + 'Text: \"' + text_one[0] + "\"" + '\n\n' + text_one[1] + '\n\n' +'Text: \"' + text_two[0] + "\"" + '\n\n' + text_two[1] + '\n\n'+'Text: \"' + text_three[0] + "\"" + '\n\n' + text_three[1] + '\n\n' + 'Text: \"' + text_four[0] + "\"" + '\n\n' + text_four[1] + '\n\n' + 'Text: \"' + text_five[0] + "\"" + '\n\n' + text_five[1] +'\n\n' + "Classify the following comment:"
# text_one
# # ['3868. Youre seriously moron 864433121', 'Label: 0']

In [56]:
with open('../../data/text/temp.txt', 'w') as f:
    f.write(f"{temp_comment}\n")

In [57]:
def check_repetition(text):
  """Counts the number of repeated words in a string."""

  words = text.lower().split()
  word_counts = Counter(words)
  repeated_words = {word: count for word, count in word_counts.items() if count > 1}
  for value in repeated_words:
    if repeated_words[value] >= 25:
      return True
  return False

In [58]:
def shot_additions(examples):
   evaluation_prompt = ''
   # Read the evaluation prompt from the text file with utf-8 encoding
   with open("../../data/text/rag_prompts/gpt-3-5-0125-two-shot-prompt.txt", "r", encoding="utf-8") as file:
        evaluation_prompt += file.read() 
   for example in examples:
       split_ex = example.split(' - ')
       comment = split_ex[0]
       label = split_ex[1]
       evaluation_prompt += '\n\nText: \"' + comment + "\"" + '\n\n' + label

   evaluation_prompt += '\n\nClassify the following comment:'

   return evaluation_prompt

In [59]:
def rag_prompt(text):
    two_shots = []
    search_results = docSearch.max_marginal_relevance_search(text, k=25, fetch_k=50)
    for i in range(len(search_results)):
        content = search_results[i].page_content
        if len(content) <= 250:
            two_shots.append(content)
        if len(two_shots) == 2:
            break
    return shot_additions(two_shots)

In [60]:
tuned = pd.DataFrame(columns=['Text', 'Toxic'])

In [61]:
evaluation_prompt = ''
# Read the evaluation prompt from the text file with utf-8 encoding
with open("../../data/text/rag_prompts/gpt-3-5-0125-two-shot-prompt.txt", "r", encoding="utf-8") as file:
    evaluation_prompt += file.read()

In [62]:
evaluation_prompt

'Classify these comments as either toxic (1) non-toxic (0). Toxic comments contain offensive, harmful, or abusive language. Label 1 for toxic and 0 for non-toxic\n\nExamples for Reference:'

In [63]:
for index, row in df.iterrows():
    text = row['Text']
    completion = fine_tuned.chat.completions.create(
        model="ft:gpt-3.5-turbo-0125:personal::ASsOrsbo",
        messages=[
            {
                "role": "system", 
                "content": rag_prompt(text)
            },
            {
                "role": "user",
                "content": text
            }
        ]
    )
    tuned.loc[index] = [row['Text'], completion.choices[0].message.content]


In [64]:
tuned

,Text,Toxic
0,largely written head problem put thoughts scre...,0
1,unable answer question7713019517,0
2,According edits add ratings based brothers poi...,0
3,take back Watch back u gwenrol whatever europe...,1
4,Take care believe bullshit see Bullshit 991842...,1
...,...,...
995,well know fuck idea notability fuck line make ...,1
996,mane fuck souja boi nigga dont go hard n paint,1
997,certainly better world service called simple N...,0
998,please interact leave warnings lie talk page c...,1


In [65]:
compare = tuned['Toxic'] == df['Toxic']
gptthreefive_original_accuracy = compare.values.sum() / compare.size

In [66]:
compare

0       True
1       True
2       True
3      False
4       True
       ...  
995     True
996     True
997     True
998     True
999     True
Name: Toxic, Length: 1000, dtype: bool

In [67]:
# GPT-4o: 97.30% with too mid-detailed evaluation prompt
# GPT-3.5 turbo 50 set: 91.80% with summarized two-shot prompt costing 73 cents
# GPT-3.5 turbo 200 set: 92.00% with summarized two-shot prompt costing 72 cents
# GPT 3.5 turbo 2000 set: 97:30% with summarized two-shot prompt costing 72 cents
print(f"Accuracy: {gptthreefive_original_accuracy * 100:.2f}%")

Accuracy: 93.60%


In [69]:
completion.choices[0].message.content

'1'